In [1]:
import pandas as pd
import numpy as np
import sys


## Import Data

In [2]:
features = pd.read_csv('../walmart-recruiting-store-sales-forecasting/features/features.csv')

train = pd.read_csv('../walmart-recruiting-store-sales-forecasting/train/train.csv')
train['Date'] = pd.to_datetime(train['Date'])

test = pd.read_csv('../walmart-recruiting-store-sales-forecasting/test/test.csv')
test['Date'] = pd.to_datetime(test['Date'])

stores = pd.read_csv('../walmart-recruiting-store-sales-forecasting/stores.csv')

In [3]:
print(features.head())
print("------------------------------------------------------------/n")
print(stores.head())
print("------------------------------------------------------------/n")
print(train.head())
print("------------------------------------------------------------/n")
print(test.head())

   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        39.93       2.514        NaN        NaN   
3      1  2010-02-26        46.63       2.561        NaN        NaN   
4      1  2010-03-05        46.50       2.625        NaN        NaN   

   MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday  
0        NaN        NaN        NaN  211.096358         8.106      False  
1        NaN        NaN        NaN  211.242170         8.106       True  
2        NaN        NaN        NaN  211.289143         8.106      False  
3        NaN        NaN        NaN  211.319643         8.106      False  
4        NaN        NaN        NaN  211.350143         8.106      False  
------------------------------------------------------------/n
   Store Type    Size
0      1    A  151315
1      2    A  202307


In [4]:
print(train.info())
print("------------------------------------------------------------/n")
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 13.3 MB
None
------------------------------------------------------------/n
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115064 entries, 0 to 115063
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Store      115064 non-null  int64         
 1   Dept       115064 non-null  int64         
 2   Date       115064 non-null  datetime64[ns]
 3   IsHoliday  115064 non-null  boo

## Data Preprocessing

In [5]:
feat_stores = features.merge(stores, how='inner', on = "Store")

feat_stores['Date'] = pd.to_datetime(feat_stores['Date'])

feat_stores['Day'] = feat_stores['Date'].dt.day
feat_stores['Week'] = feat_stores['Date'].dt.week
feat_stores['Month'] = feat_stores['Date'].dt.month
feat_stores['Year'] = feat_stores['Date'].dt.year
feat_stores['WeekOfYear'] = (feat_stores.Date.dt.isocalendar().week)*1.0 

train_df = train.merge(feat_stores, how='inner', on = ['Store','Date','IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)
test_df = test.merge(feat_stores, how='inner', on = ['Store','Date','IsHoliday']).sort_values(by = ['Store','Dept','Date']).reset_index(drop=True)

print(train_df.head())
print("------------------------------------------------------------/n")
print(test_df.head())

C:\Users\brock\AppData\Local\Temp\ipykernel_95744\1984007329.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  feat_stores['Week'] = feat_stores['Date'].dt.week


   Store  Dept       Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0      1     1 2010-02-05      24924.50      False        42.31       2.572   
1      1     1 2010-02-12      46039.49       True        38.51       2.548   
2      1     1 2010-02-19      41595.55      False        39.93       2.514   
3      1     1 2010-02-26      19403.54      False        46.63       2.561   
4      1     1 2010-03-05      21827.90      False        46.50       2.625   

   MarkDown1  MarkDown2  MarkDown3  ...  MarkDown5         CPI  Unemployment  \
0        NaN        NaN        NaN  ...        NaN  211.096358         8.106   
1        NaN        NaN        NaN  ...        NaN  211.242170         8.106   
2        NaN        NaN        NaN  ...        NaN  211.289143         8.106   
3        NaN        NaN        NaN  ...        NaN  211.319643         8.106   
4        NaN        NaN        NaN  ...        NaN  211.350143         8.106   

   Type    Size  Day  Week  Month  Year  Wee

In [6]:
print(train_df.info())
print("------------------------------------------------------------/n")
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
 5   Temperature   421570 non-null  float64       
 6   Fuel_Price    421570 non-null  float64       
 7   MarkDown1     150681 non-null  float64       
 8   MarkDown2     111248 non-null  float64       
 9   MarkDown3     137091 non-null  float64       
 10  MarkDown4     134967 non-null  float64       
 11  MarkDown5     151432 non-null  float64       
 12  CPI           421570 non-null  float64       
 13  Unemployment  421570 non-null  float64       
 14  Type          421570 non-null  object        
 15  Size          421

## Feature Engineering

Feature Dropping, Scaling, and Splitting

Explaination for which features we drop can be found in the EDA Notebook

In [7]:
# Drop Features
interm_train = train_df.drop(['Date', 'Day', 'Month', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Type']
                             , axis=1)

target = test_df.drop(['Date', 'Day', 'Month', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Type']
                      , axis=1)

# Fill NaN

interm_train=interm_train.fillna(0)
target=target.fillna(0)

In [8]:
# Build input and target
train_cols = interm_train.columns.to_list()
train_cols.remove('Weekly_Sales')
target_col = 'Weekly_Sales'

train_df = interm_train[train_cols].copy()
test_df = interm_train[target_col].copy()

In [9]:
print(train_df.info())
print("------------------------------------------------------------/n")
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Store       421570 non-null  int64  
 1   Dept        421570 non-null  int64  
 2   IsHoliday   421570 non-null  bool   
 3   MarkDown1   421570 non-null  float64
 4   MarkDown2   421570 non-null  float64
 5   MarkDown3   421570 non-null  float64
 6   MarkDown4   421570 non-null  float64
 7   MarkDown5   421570 non-null  float64
 8   Size        421570 non-null  int64  
 9   Week        421570 non-null  int64  
 10  Year        421570 non-null  int64  
 11  WeekOfYear  421570 non-null  Float64
dtypes: Float64(1), bool(1), float64(5), int64(5)
memory usage: 36.2 MB
None
------------------------------------------------------------/n
<class 'pandas.core.series.Series'>
RangeIndex: 421570 entries, 0 to 421569
Series name: Weekly_Sales
Non-Null Count   Dtype  
--------------   -----  
421570 no

In [10]:
# Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(interm_train[train_cols])
train_df_nonstd = train_df
train_df[train_cols] = scaler.transform(train_df[train_cols])
target[train_cols] = scaler.transform(target[train_cols])

In [11]:
print(train_df.head())
print("------------------------------------------------------------/n")
print(target.head())
print("------------------------------------------------------------/n")
print(test_df.head())

   Store  Dept  IsHoliday  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
0    0.0   0.0        0.0        0.0   0.002536   0.000205        0.0   
1    0.0   0.0        1.0        0.0   0.002536   0.000205        0.0   
2    0.0   0.0        0.0        0.0   0.002536   0.000205        0.0   
3    0.0   0.0        0.0        0.0   0.002536   0.000205        0.0   
4    0.0   0.0        0.0        0.0   0.002536   0.000205        0.0   

   MarkDown5      Size      Week  Year  WeekOfYear  
0        0.0  0.630267  0.078431   0.0    0.078431  
1        0.0  0.630267  0.098039   0.0    0.098039  
2        0.0  0.630267  0.117647   0.0    0.117647  
3        0.0  0.630267  0.137255   0.0    0.137255  
4        0.0  0.630267  0.156863   0.0    0.156863  
------------------------------------------------------------/n
   Store  Dept  IsHoliday  MarkDown1  MarkDown2  MarkDown3  MarkDown4  \
0    0.0   0.0        0.0   0.076330   0.051662   0.000564   0.053945   
1    0.0   0.0        0.0   0.1288

In [12]:
# Split Data
from sklearn.model_selection import train_test_split

X_train_std, X_test_std, y_train, y_test = train_test_split(
    train_df, test_df, test_size=0.3, random_state=1
)

X_train, X_test, y_train, y_test = train_test_split(
    train_df_nonstd, test_df, test_size=0.3, random_state=1
)

In [13]:

# Define the function to evaluate the models

def WMAE(df, targets, predictions):
    weights = df.IsHoliday.apply(lambda x: 5 if x else 1)
    return np.round(np.sum(weights*abs(targets-predictions))/(np.sum(weights)), 2)



In [14]:
X_train.info()

X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295099 entries, 64010 to 128037
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Store       295099 non-null  float64
 1   Dept        295099 non-null  float64
 2   IsHoliday   295099 non-null  float64
 3   MarkDown1   295099 non-null  float64
 4   MarkDown2   295099 non-null  float64
 5   MarkDown3   295099 non-null  float64
 6   MarkDown4   295099 non-null  float64
 7   MarkDown5   295099 non-null  float64
 8   Size        295099 non-null  float64
 9   Week        295099 non-null  float64
 10  Year        295099 non-null  float64
 11  WeekOfYear  295099 non-null  float64
dtypes: float64(12)
memory usage: 29.3 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 126471 entries, 367951 to 342931
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Store       126471 non-null  float64
 1   Dept    

## Linear Regression

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import time

lr = LinearRegression()
st = time.time()
lr.fit(X_train, y_train)
en = time.time()
fit_time = (en-st)

pred_train = lr.predict(X_train)
pred_test = lr.predict(X_test)

lr = LinearRegression()
st = time.time()
lr.fit(X_train_std, y_train)
en = time.time()
fit_time_std = (en-st)

pred_train_std = lr.predict(X_train_std)
pred_test_std = lr.predict(X_test_std)

## R2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)

r2_train_std = r2_score(y_train, pred_train_std)
r2_test_std = r2_score(y_test, pred_test_std)

## TIME
print('Fit Time: ', fit_time)
print('Fit Time (STD):', fit_time_std)

print("R2 Scores\n---------------------\n")
print("R2 (train) for %s: %0.5f%% " % ('LR', r2_train))
print("R2 (test) for %s: %0.5f%% " % ('LR', r2_test))
print("R2 (train - STD) for %s: %0.5f%% " % ('LR', r2_train_std))
print("R2 (test - STD) for %s: %0.5f%% \n\n" % ('LR', r2_test_std))

## MSE
mse_train = mean_squared_error(y_train, pred_train)
mse_test = mean_squared_error(y_test, pred_test)
mse_train_std = mean_squared_error(y_train, pred_train_std)
mse_test_std = mean_squared_error(y_test, pred_test_std)

print("MSE Scores\n---------------------\n")
print("MSE (train) for %s: %0.5f%% " % ('LR', mse_train))
print("MSE (test) for %s: %0.5f%% " % ('LR', mse_test))
print("MSE (train - STD) for %s: %0.5f%% " % ('LR', mse_train_std))
print("MSE (test - STD) for %s: %0.5f%% " % ('LR', mse_test_std))

Fit Time:  0.1607663631439209
Fit Time (STD): 0.1431887149810791
R2 Scores
---------------------

R2 (train) for LR: 0.08507% 
R2 (test) for LR: 0.08605% 
R2 (train - STD) for LR: 0.08507% 
R2 (test - STD) for LR: 0.08605% 


MSE Scores
---------------------

MSE (train) for LR: 470713421.55355% 
MSE (test) for LR: 474215398.63295% 
MSE (train - STD) for LR: 470713421.55355% 
MSE (test - STD) for LR: 474215398.63295% 


## Ridge Regression

In [16]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error
import time

lr = Ridge()
st = time.time()
lr.fit(X_train, y_train)
en = time.time()
fit_time = (en-st)

pred_train = lr.predict(X_train)
pred_test = lr.predict(X_test)

lr = Ridge()
st = time.time()
lr.fit(X_train_std, y_train)
en = time.time()
fit_time_std = (en-st)

pred_train_std = lr.predict(X_train_std)
pred_test_std = lr.predict(X_test_std)

## R2
r2_train = r2_score(y_train, pred_train)
r2_test = r2_score(y_test, pred_test)

r2_train_std = r2_score(y_train, pred_train_std)
r2_test_std = r2_score(y_test, pred_test_std)

## TIME
print('Fit Time: ', fit_time)
print('Fit Time (STD):', fit_time_std)

print("R2 Scores\n---------------------\n")
print("R2 (train) for %s: %0.5f%% " % ('LR', r2_train))
print("R2 (test) for %s: %0.5f%% " % ('LR', r2_test))
print("R2 (train - STD) for %s: %0.5f%% " % ('LR', r2_train_std))
print("R2 (test - STD) for %s: %0.5f%% \n\n" % ('LR', r2_test_std))

## MSE
mse_train = mean_squared_error(y_train, pred_train)
mse_test = mean_squared_error(y_test, pred_test)
mse_train_std = mean_squared_error(y_train, pred_train_std)
mse_test_std = mean_squared_error(y_test, pred_test_std)

print("MSE Scores\n---------------------\n")
print("MSE (train) for %s: %0.5f%% " % ('LR', mse_train))
print("MSE (test) for %s: %0.5f%% " % ('LR', mse_test))
print("MSE (train - STD) for %s: %0.5f%% " % ('LR', mse_train_std))
print("MSE (test - STD) for %s: %0.5f%% " % ('LR', mse_test_std))

Fit Time:  0.03339529037475586
Fit Time (STD): 0.024975299835205078
R2 Scores
---------------------

R2 (train) for LR: 0.08507% 
R2 (test) for LR: 0.08605% 
R2 (train - STD) for LR: 0.08507% 
R2 (test - STD) for LR: 0.08605% 


MSE Scores
---------------------

MSE (train) for LR: 470713465.40145% 
MSE (test) for LR: 474214913.87595% 
MSE (train - STD) for LR: 470713465.40145% 
MSE (test - STD) for LR: 474214913.87595% 
